<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title"><b>Capacitated Vehicle Routing Problem</b></span> by <a xmlns:cc="http://creativecommons.org/ns#" href="http://mate.unipv.it/gualandi" property="cc:attributionName" rel="cc:attributionURL">Stefano Gualandi</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/mathcoding/opt4ds" rel="dct:source">https://github.com/mathcoding/opt4ds</a>.

**NOTE:** Run the following script whenever running this script on a Google Colab.

In [ ]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("glpk") or os.path.isfile("glpk")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge glpk 
        except:
            pass

# Capacitated Vehicle Routing Problem
In this lab session, you have to find a solution for the Capacitated Vehicle Routing Problem, likely, but not necessarily, by using an **Integer Linear Programming** model.

The problem is defined as follows. You are given a set of $n$ customers, and each customer has a demand of $d_i$ unit of product. The position of each customer is given by a pair of coordinates: in this exercise, they are points in an Euclidean plane. The distance between any pair of customers is equal to the Euclidean distance between the corresponding pair of points. You are give also a set of $k$ vehicles with capacity $C$, such that, $d_i \leq C, i=1,\dots,n$. All the vehicles are located at the same depot.

The problem is to find a routing of the vehicle that minimize the overall number of km travelled by the vehicles, while respecting the load capacity of each vehicle.

## Helper functions
To facilitate the modeling activity, the following functions are provided with this notebook:

1. A parse function to read an instance of CVRP with a fix format.

2. A function to compute the distance between a pair of points.

2. A function to plot a solution. You can use this function *manually* to try to see possible simple solutions. The `PlotSolution(Xs, Ws, Es)` takes in input a list of coordinates of the customers (the first point is always the depot), a list of demands, and a list of link connecting the points (the last list can be set manually).

Those functions are given below.

In [ ]:
def ParseFile(filename):
    doc = open(filename, 'r')
    # Salta le prime 3 righe
    for _ in range(3):
        doc.readline()
    # Leggi la dimensione
    n = int(doc.readline().split(' ')[2])
    # sala riga
    doc.readline()
    # Leggi la capacita
    C = int(doc.readline().split(' ')[2])
    # sala riga
    doc.readline()
    # Leggi posizioni
    Ps = {}
    for row in doc:
        row = row.rstrip().split(' ')
        if row[0] == 'DEMAND_SECTION':
            break
        row = list(map(lambda z: int(z), row))
        Ps[row[0]] = (row[1], row[2])
    # Leggi posizioni
    Ds = {}
    for row in doc:
        row = row.rstrip().split(' ')
        if row[0] == 'DEPOT_SECTION':
            break
        row = list(map(lambda z: int(z), row))
        Ds[row[0]] = row[1]
    d = int(next(doc).rstrip())

    return n, C, Ps, Ds, d

In [ ]:
def Distance(A, B):
    return math.sqrt((A[0]-B[0])**2 + (A[1] - B[1])**2)

In [ ]:
def DisegnaPunto(A):
    """
    Disegna un punto nel piano
    """
    plt.plot([A[0]], [A[1]], 'bo', alpha=0.5)
    
def DisegnaSegmento(A, B):
    """ 
    Disegna un segmento nel piano dal punto A a al punto B
    Vedi manuale a: http://matplotlib.org/api/pyplot_api.html
    """
    # Disegna il segmento
    plt.plot([A[0], B[0]], [A[1], B[1]], 'b', lw=0.75)
    # Disegna gli estremi del segmento
    DisegnaPunto(A)
    DisegnaPunto(B)

In [ ]:
import matplotlib.pyplot as plt

def PlotSolution(Xs, Ws, Es):
    plt.figure(figsize=(6,6))
    for i,j in Es:
        DisegnaSegmento(Ps[i], Ps[j])
    plt.scatter([i for i,j in Xs[1:]], [j for i,j in Xs[1:]], 
                s=Ws[1:], alpha=0.3, cmap='viridis')
    plt.plot([Xs[0][0]], [Xs[0][1]], marker='s', color='red', alpha=0.5)
    plt.axis('square')
    plt.axis('off')

The following snippet use those functions to parse the file `E-n23-k3.vrp` and to plot the instance.

In [ ]:
!wget http://www-dimat.unipv.it/gualandi/opt4ds/E-n23-k3.vrp

In [ ]:
n, C, Ps, Ds, d = ParseFile("../data/E-n23-k3.vrp")

Xs = [p for p in Ps.values()]
Ws = [w for w in Ds.values()]

PlotSolution(Xs, Ws, [(1,2),(2,3),(3,1), (1,10), (10,1)])
print(d)

In [ ]:
PlotSolution(Xs, Ws, [(1,10),(10,1), (1,2),(2,3),(3,1)])

## Scaling up the size of the instances
You can try your solution approach with the following instances taken from this website (http://vrp.galgos.inf.puc-rio.br/index.php/en/):

1. E-n22-k4.vrp
2. E-n23-k3.vrp
3. E-n30-k3.vrp  
4. E-n33-k4.vrp  
5. E-n51-k5.vrp 
6. E-n101-k8.vrp   
7. X-n101-k25.vrp

**NOTE:** Start with the smaller instances, and when scaling to larger instances, fix a timeout.

### Computation Challenge
If you are passionate about this type of computational challenges, you can try to participate to the:

[12th DIMACS Implementation Challenge](http://dimacs.rutgers.edu/programs/challenge/vrp/)

This is a possible project for the final exam of this course.

## [Optional] Capacitated VRP with Time Windows
Once you have a satisfactory solution for the CVRP consider the following more realistic variant:

Every customer can be visited only within a given time window $[e_i, l_i], i=2,\dots,n$, and it has a service time $s_i$. A vehicle can arrive before time $e_i$, but the service will not start before $s_i$.

Find the routing minimizing the overall time to serve all customers, while respecting the time windows constraints.
In this case, the distance between a pair of customers is given in traveling time instead of km.